In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from mmnl import MMNL

In [3]:
d = pd.read_csv('./data/catsup_trainformat.csv', delimiter=',')
d.head(10)

,chosen,id,alt,display,feature,price,chid
0,0,1,0,0,0,4.6,0
1,0,1,1,0,0,3.7,0
2,1,1,2,0,0,5.2,0
3,0,1,3,0,0,3.4,0
4,0,1,0,0,0,4.6,1
5,0,1,1,0,0,4.3,1
6,1,1,2,0,0,5.2,1
7,0,1,3,0,0,4.4,1
8,0,1,0,0,0,4.6,2
9,0,1,1,0,1,2.5,2


In [4]:
X = d.values[:,1:-1]
X.shape

(11192, 5)

In [5]:
def summarize_data(model): 
    #model should be in method(draws) format
    result = []
    for file in os.listdir('.\\resultaten\\%s'%(model)):
        if file.endswith('%s_utsgdp_results.p'%(model)):
            infile = open(os.path.join('.\\resultaten\\%s'%(model), file), 'rb')
            opt_list = pickle.load(infile)
            [result.append(i) for i in opt_list]
    
    pickle.dump( result, open( "./resultaten/%s/mcdgp%s_utsdgp_all_results.p" %(model,model), "wb" ) )
    return result      
        

In [6]:
def probs(X,model,theta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    method = model.split('(')[0]
    R = int(model.split('(')[1][:-1])
    print(R)
    if method == 'QMC':
        delta = QMC(300,3,R)
    elif method == 'SMC':
        delta = np.random.standard_normal((300,3,R))
    
    beta = theta[:3][:, None] + delta * np.abs(theta[3:][:, None])
    try:
        assert(X.shape == (11192, 5) and beta.shape == (300,3,R))
        
    except AssertionError:
        raise AssertionError('Ga X ff in juiste format gooien. X: %s, beta: %s' %(X.shape,beta.shape))
    beta_choice = np.zeros((3,11192,R))
    for i in range(11192):
        id = int(X[i,0])
        beta_choice[:,i,:] = beta[id-1,:,:]
        
    P = np.zeros((11192,R))
    for r in range(R):
        P[:,r] = np.exp((X[:,2:]@ beta_choice[:,:,r])[:,0])
        
    try:
        assert(P.shape == (11192,R) or P.shape == (R,11192))
    except AssertionError:
        raise AssertionError('Product van X en beta gaat niet goed, P.shape is nu %g' % (P.shape))
#     try:
#         p = P[0:4]
#         check = p/np.sum(p)
#         assert(np.sum(check) == 1. or np.sum(check) == 1)
#     except AssertionError:
#         raise AssertionError('Kansen van eerste aankoop sommeren niet naar 1 %f'%(np.sum(check)))

    Y = np.zeros(11192,)
    for i in range(0,11192, 4):
        sum = np.sum(P[i:i+4,:],axis=0)
        Y[i:i+4] = np.mean(P[i:i+4]/sum,axis=1)
#         p = P[i:i+4]
#         check = p/np.sum(p)
#         print(np.sum(check))
    return Y
# Y_true2 = probs(X,'QMC(1000)',np.array([1.5,1,-1.1,0.8,0.1,1.2]))

In [30]:
def get_results(model_data,method,Y_true=None):
    #model data should be in list format containing OptimizeResult types
    theta_true = np.array([1.5,  1.,  -1.1,  0.4,  0.1,  0.6])
    results ={}
    theta_est = np.zeros((6,))
    mape = 0
    D = len(model_data)
    print(D)
    for run in model_data:
        theta_est[:3] += run.x[:3]/D
        t = np.exp(run.x[3:])/D
        if np.any(np.isinf(t)):
            print(run)
        else:
            theta_est[3:] += t
#     P = probs(X,method,theta_est)
#     mape_choice = 100*np.sum(np.abs((Y_true-P)/Y_true))/11192
#     rmse_choice = np.sqrt(np.sum((Y_true-P)**2)/11192)
    mape_par = 100*np.sum(np.abs((theta_true-theta_est)/theta_true))/6
    rmse_par = np.sqrt(np.sum((theta_true-theta_est)**2)/6)

    results = { 'method': method,
                'theta': theta_est,
#                 'choice': {'mape': mape_choice,
#                            'rmse': rmse_choice,
#                           },
               'Parameter': {'mape': mape_par,
                             'rmse': rmse_par
                           }
            }
    return results

Get true probabilities based on big QMC model

In [8]:
smc250 = summarize_data('SMC(250)')
len(smc250)

100

In [31]:
ressmc250 = get_results(smc250,'SMC(250)')
print(ressmc250)

100
      fun: nan
 hess_inv: array([[ 7.95384472e-03, -2.10314516e-03, -1.34707456e-03,
         1.80204711e-01, -5.01558323e-01,  1.07650173e-01],
       [-2.10314516e-03,  1.06484684e-02,  4.94953765e-03,
        -3.89576990e-01,  1.13254738e+00, -2.33998988e-01],
       [-1.34707456e-03,  4.94953765e-03,  1.48310697e-02,
        -1.24514052e+00,  3.57904416e+00, -7.45988794e-01],
       [ 1.80204711e-01, -3.89576990e-01, -1.24514052e+00,
         1.09046961e+02, -3.09368344e+02,  6.55162465e+01],
       [-5.01558323e-01,  1.13254738e+00,  3.57904416e+00,
        -3.09368344e+02,  8.83437028e+02, -1.85740859e+02],
       [ 1.07650173e-01, -2.33998988e-01, -7.45988794e-01,
         6.55162465e+01, -1.85740859e+02,  3.94198540e+01]])
      jac: array([nan, nan, nan, nan, nan, nan])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 1160
      nit: 22
     njev: 145
   status: 2
  success: False
        x: array([-4.95544184e+00,  2.00668987e+01,  5.2

In [14]:
qmc25 = summarize_data('QMC(25)')
len(qmc25)

100

In [16]:
resqmc25 = get_results(qmc25,'QMC(25)',Y_true)
resqmc25

{'method': 'QMC(25)',
 'theta': array([ 0.96361293,  0.8029465 , -1.11864207,  0.70108456,  0.46697882,
         0.03907107]),
 'Parameter': {'mape': 88.8744240008519, 'rmse': 0.550621223547447}}

In [11]:
model = 'SMC(500)'
for file in os.listdir('.\\resultaten\\%s'%(model)):
    if file.endswith('%s_results.p'%(model)):
        infile = open(os.path.join('.\\resultaten\\%s'%(model), file), 'rb')
        opt_list = pickle.load(infile)
        print(len(opt_list),file)

100 0-100_SMC(500)_results.p
